# Try GRASS GIS in Jupyter Notebook with Bash

[<img src="https://grass.osgeo.org/uploads/images/logo/grassgis_logo_colorlogo_text_alphabg.png" alt="GRASS GIS" style="width:200px;"/>](https://grass.osgeo.org/)

This is a quick introduction into *GRASS GIS* in a *Jupyter Notebook* using a *Bash* command line.
The interactive notebook is available online thanks to the [*Binder*](https://mybinder.org/) service.

The GRASS GIS is already running in the background, so you can
execute GRASS GIS commands (modules) right away.
Examples here are using a sample GRASS GIS dataset for North Carolina, USA.
The dataset is included in this environment.

## Usage

To run the selected part which is called a cell, hit `Shift + Enter`.

## Session  and configuration

There is several ways how GRASS GIS can be used. When you start GRASS GIS on a desktop,
besides grapical user intrface (GUI), you can also a command line (CLI).
This would be typically Bash on Linux other other similar systems.
In this command line, you would be able to execute GRASS GIS commands (modules)
which are only available within GRASS GIS session.
This notebook is already running inside a GRASS GIS session which is connected
to a database containing the dataset mentioned above.
In addition to it, we set here several variables useful for using GRASS GIS in a notebook.

In [1]:
# simply overwrite existing maps like we overwrite Python variable values
export GRASS_OVERWRITE=1
# enable map rendering to in Jupyter Notebook
export GRASS_FONT=sans
# set display modules to render into a file (named map.png by default)
export GRASS_RENDER_IMMEDIATE=cairo
export GRASS_RENDER_FILE_READ=TRUE
export GRASS_LEGEND_FILE=legend.txt

## Raster buffer

Set computational region and create multiple buffers in given distances
around lakes represented as raster:

In [2]:
g.region raster=lakes -pg
r.buffer input=lakes output=lakes_buff distances=60,120,240,500
d.erase
d.rast map=lakes_buff
d.legend raster=lakes_buff range=2,5 at=80,100,2,10

projection=99
zone=0
n=228500
s=215000
w=630000
e=645000
nsres=10
ewres=10
rows=1350
cols=1500
cells=2025000
Reading input raster map <lakes@PERMANENT>...
   0%   3%   6%   9%  12%  15%  18%  21%  24%  27%  30%  33%  36%  39%  42%  45%  48%  51%  54%  57%  60%  63%  66%  69%  72%  75%  78%  81%  84%  87%  90%  93%  96%  99% 100%
Finding buffer zones...
   0%   3%   6%   9%  12%  15%  18%  21%  24%  27%  30%  33%  36%  39%  42%  45%  48%  51%  54%  57%  60%  63%  66%  69%  72%  75%  78%  81%  84%  87%  90%  93%  96%  99% 100%
Writing output raster map <lakes_buff>...
   0%   3%   6%   9%  12%  15%  18%  21%

In [3]:
%html <img src="map.png">
# you may need to reload the page
# when you render a new image to update it

## Vector buffer

Create a negative buffer around state boundary represented as a vector.
Vector modules typically don't follow computational region,
but we set it to inform display modules about our area of interest.

In [4]:
v.buffer input=boundary_state output=buffer distance=-10000
g.region vector=boundary_state -pg
# we need to use a unique name for every image because web browser caches images by name
export GRASS_RENDER_FILE=vector.png
d.erase  # erase the display before drawing again
rm -f $GRASS_LEGEND_FILE  # and remove the legend file
d.vect map=boundary_state fill_color="#5A91ED", legend_label="State boundary"
d.vect map=buffer fill_color="#F8766D" legend_label="Inner portion"
d.legend.vect at=10,35

Buffering areas...
  12%  25%WARNING: No coordinates in GEOS geometry (can be ok for negative distance)!
  37%WARNING: No coordinates in GEOS geometry (can be ok for negative distance)!
  50%WARNING: No coordinates in GEOS geometry (can be ok for negative distance)!
  62%WARNING: No coordinates in GEOS geometry (can be ok for negative distance)!
  75%WARNING: No coordinates in GEOS geometry (can be ok for negative distance)!
  87%WARNING: No coordinates in GEOS geometry (can be ok for negative distance)!
 100%
Cleaning buffers...
Building parts of topology...
Building topology for vector map <buffer@user1>...
Registering primitives...
2 primitives registered
1588 vertices registered
Number of nodes: 2
Number of primitives: 2
Number of points: 0
Number of lines: 0
Number of boundaries: 2
Number of centroids: 0
Number of areas: -
Number of isles: -
Snapping boundaries...
Reading features...
Snap vertices Pass 1: select points
   0%  50%

In [5]:
%html <img src="vector.png">

### testing las data import and clip

In [ ]:
las2las points.las --output points_clip.las \
    --extent "2090835 730212 2092347 731322"

In [ ]:
7zr a points.las.7z points.las

## Testing import imagery and compress it

### Orthophoto

#### the geotiff was imported as ortho object

In [8]:
r.import input=ChilloweLakeObjectLC.tif output=ortho


         Location PROJ_INFO is:
         name: Lambert Conformal Conic
         proj: lcc
         datum: nad83
         a: 6378137.0
         es: 0.006694380022900787
         lat_1: 36.16666666666666
         lat_2: 34.33333333333334
         lat_0: 33.75
         lon_0: -79
         x_0: 609601.22
         y_0: 0
         no_defs: defined

         Dataset PROJ_INFO is:
         name: WGS 84 / UTM zone 12N
         datum: wgs84
         ellps: wgs84
         proj: utm
         zone: 12
         no_defs: defined

         ERROR: proj

         In case of no significant differences in the projection
         definitions, use the -o flag to ignore them and use current
         location definition.
         Consider generating a new location from the input dataset using
         the 'location' parameter.
Importing raster map <ortho>...
   0%   3%   6%   9%  12%  15%  18%  21%  24%  27%  30%  33%  36%  39%  42% 

#### The ortophoto raster map in GRASS GIS was compressed using BZIP2:

In [9]:
GRASS_COMPRESSOR=BZIP2 r.compress map=ortho

Re-compressing <ortho>
   0%   3%   6%   9%  12%  15%  18%  21%  24%  27%  30%  33%  36%  39%  42%  45%  48%  51%  54%  57%  60%  63%  66%  69%  72%  75%  78%  81%  84%  87%  90%  93%  96%  99%DONE: compressed file is 18 KB (14.63%) larger



#### and exported in (native) GRASS GIS pack format:

In [10]:
r.pack input=ortho output=ortho.grpack

         overwritten
Packing <ortho@user1> to </home/jovyan/ortho.grpack>...



### Unpack a raster package

In [12]:
r.unpack input=ortho.grpack output=Newortho -o

WARNING: Overriding projection check (using current location's projection).
Raster map <Newortho> unpacked



In [6]:
 r.unpack input=NCortho.grpack output=NCortho

         location:
         + name: Lambert Conformal Conic
         - name: unnamed
         - ellps: grs80
         proj: lcc
         + datum: nad83
         + a: 6378137.0
         + es: 0.006694380022900787
         lat_1: 36.16666666666666
         lat_2: 34.33333333333334
         lat_0: 33.75
         lon_0: -79
         - x_0: 609601.2192024384
         ? -- ------
         + x_0: 609601.22
         y_0: 0
         no_defs: defined
WARNING: Difference between PROJ_UNITS file of packed map and of current
         location:
         - unit: Foot_US
         - units: Foot_USs
         - meters: 0.3048006096012192
         + unit: Meter
         + units: Meters
         + meters: 1
ERROR: Projection of dataset does not appear to match current location. In
       case of no significant differences in the projection definitions,
       use the -o flag to ignore them and use current location definition.



## Zones

#### Manually selected and digitized polygons for zonal analysis were exported from GRASS GIS in a (native) ASCII format using:

In [ ]:
v.out.ascii input=zones layer=-1 type=boundary,centroid \
    format=standard precision=2 output=zones.txt

#### Trailing spaces were removed using:

In [ ]:
sed -i 's/\s*$//g' zones.txt
The header was edited and cleaned up manually.